In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
# # mergedf = pd.read_csv('metadata_otu_merged_famID-to-str.csv')
# mergedf = pd.read_csv('metadata_otu_merged_non-rarefied.csv')
# # mergedf = pd.read_csv('metadata_otu_merged_famID-to-str_onlyFamwPet.csv')

# mergedf.head(1)

mergedf = pd.read_csv('metadata_otu_merged_famID-to-str.csv', low_memory=False)
# mergedf = pd.read_csv('metadata_otu_merged_famID-to-str_onlyFamwPet.csv', low_memory=False)
# mergedf = pd.read_csv('metadata_otu_merged_non-rarefied.csv', low_memory=False)
dog_fam = pd.read_csv('dogFamily_toMerge.csv')
mergedf = mergedf.drop("Unnamed: 0", axis=1)
mergedf.head(1)

,sample_name,age,age_in_years,age_units,agegroup,anonymized_name,body_habitat,body_product,body_site,collection_timestamp,...,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__Nocardioidaceae; g__; s__.16,k__Bacteria; p__Firmicutes; c__Erysipelotrichi; o__Erysipelotrichales; f__Erysipelotrichaceae; g__; s__.7,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__[Odoribacteraceae]; g__Odoribacter; s__.1,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__intermedia,k__Bacteria; p__Firmicutes; c__Bacilli; o__Lactobacillales; f__Streptococcaceae; g__Streptococcus; s__.41,k__Bacteria; p__Cyanobacteria; c__Chloroplast; o__Streptophyta; f__; g__; s__.6,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.54,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__.23,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.55,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__Propionibacteriaceae; g__Propionibacterium; s__granulosum
0,797.02.HA.Forehead,1,1,years,Adolescent,FS_01,UBERON:skin,UBERON:sebum,UBERON:face,4/6/2010,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#MERGE with DOG_FAM info
mergedf = mergedf.merge(dog_fam, left_on="family", right_on="family", suffixes=(False,False), how="outer")
mergedf["have_dog"].fillna("no",inplace = True)
print(mergedf.shape)
mergedf.head()

(1050, 1088)


,sample_name,age,age_in_years,age_units,agegroup,anonymized_name,body_habitat,body_product,body_site,collection_timestamp,...,k__Bacteria; p__Firmicutes; c__Erysipelotrichi; o__Erysipelotrichales; f__Erysipelotrichaceae; g__; s__.7,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__[Odoribacteraceae]; g__Odoribacter; s__.1,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__intermedia,k__Bacteria; p__Firmicutes; c__Bacilli; o__Lactobacillales; f__Streptococcaceae; g__Streptococcus; s__.41,k__Bacteria; p__Cyanobacteria; c__Chloroplast; o__Streptophyta; f__; g__; s__.6,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.54,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__.23,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.55,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__Propionibacteriaceae; g__Propionibacterium; s__granulosum,have_dog
0,797.02.HA.Forehead,1,1,years,Adolescent,FS_01,UBERON:skin,UBERON:sebum,UBERON:face,4/6/2010,...,0,0,0,0,0,0,0,0,0,no
1,797.02.HA.LeftPalm,1,1,years,Adolescent,FS_01,UBERON:skin,UBERON:sebum,UBERON:skin of hand,4/6/2010,...,0,0,0,5,0,0,0,0,0,no
2,797.02.HA.RightPalm,1,1,years,Adolescent,FS_01,UBERON:skin,UBERON:sebum,UBERON:skin of hand,4/6/2010,...,0,0,0,0,0,0,0,0,0,no
3,797.02.HA.Stool,1,1,years,Adolescent,FS_01,UBERON:feces,UBERON:feces,UBERON:feces,4/6/2010,...,0,3,0,0,0,0,0,5,0,no
4,797.02.HA.Tongue,1,1,years,Adolescent,FS_01,UBERON:oral cavity,UBERON:saliva,UBERON:tongue,4/6/2010,...,0,0,0,0,0,0,0,0,0,no


# FILTERING Train and Test Datasets

In [4]:
#FILTER 1
only_humans_with_dogs = 1

#FILTER 2
# sample_type = "skin"
#FILTER 3
not_sample_type = "stool"

# #FILTER 4
human_role = "Partner"
#FILTER 5
# not_human_role = "Offspring"

#FILTER 6
# not_familyID = 63

# SPLITTING TRAIN and TEST (test = dogs, train = humans)

In [5]:
human_data_train = mergedf[mergedf.host_common_name =="human"]

dog_data_test = mergedf[mergedf.host_common_name =="dog"]
dog_data_test.head(1)

,sample_name,age,age_in_years,age_units,agegroup,anonymized_name,body_habitat,body_product,body_site,collection_timestamp,...,k__Bacteria; p__Firmicutes; c__Erysipelotrichi; o__Erysipelotrichales; f__Erysipelotrichaceae; g__; s__.7,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__[Odoribacteraceae]; g__Odoribacter; s__.1,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__intermedia,k__Bacteria; p__Firmicutes; c__Bacilli; o__Lactobacillales; f__Streptococcaceae; g__Streptococcus; s__.41,k__Bacteria; p__Cyanobacteria; c__Chloroplast; o__Streptophyta; f__; g__; s__.6,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.54,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__.23,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.55,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__Propionibacteriaceae; g__Propionibacterium; s__granulosum,have_dog
20,797.03.CA.BackLeftPaw,Not recorded,Not recorded,years,Not recorded,FS_175,UBERON:skin,UBERON:sebum,UBERON:foot skin,2/10/2010,...,0,0,0,0,0,0,0,0,0,yes


In [6]:
# Applying filters above
try:
    if only_humans_with_dogs == 1:
            human_data_train = human_data_train[human_data_train.have_dog =="yes"]
    
    elif only_humans_with_dogs ==0:
            human_data_train = human_data_train[human_data_train.have_dog =="no"]
except:
    print("no filter here 1")

try:    
    if sample_type:
        human_data_train = human_data_train[human_data_train.sample_type ==sample_type]
        dog_data_test = dog_data_test[dog_data_test.sample_type ==sample_type]
except:
    print("no filter here 2")
    
try:    
    if not_sample_type:
        human_data_train = human_data_train[human_data_train.sample_type !=not_sample_type]
        dog_data_test = dog_data_test[dog_data_test.sample_type !=not_sample_type]
except:
    print("no filter here 3")
    
try:    
    if human_role:
        human_data_train = human_data_train[human_data_train.role ==human_role]
except:
    print("no filter here 4")
    
try:
    if not_human_role:
        human_data_train = human_data_train[human_data_train.role !=not_human_role]    
except:
    print("no filter here 5")
    
try:    
    if not_familyID:
        human_data_train = human_data_train[human_data_train.role !=not_familyID]
        dog_data_test = dog_data_test[dog_data_test.role !=not_familyID]  
except:
    print("no filter here 6")

no filter here 2
no filter here 5
no filter here 6


# Train/test split

In [7]:
X_train = human_data_train.iloc[:, -1035:-1]
y_train = human_data_train["familyID"].values.reshape(-1, 1)

y_test = dog_data_test["familyID"].values.reshape(-1, 1)
X_test = dog_data_test.iloc[:, -1035:-1]

# Scaled Model Fitting

In [9]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
model = classifier.fit(X_train_scaled, y_train.ravel())

In [12]:
print(f'Test Data Model Score: {round(100*(classifier.score(X_test_scaled, y_test)),2)}%')
print("------------")
predictions = classifier.predict(X_test_scaled)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

Test Data Model Score: 16.38%
------------
First 10 Predictions:   ['family#71' 'family#71' 'family#71' 'family#3' 'family#71' 'family#64'
 'family#49' 'family#7' 'family#17' 'family#27']
First 10 Actual labels: [['family#3'], ['family#3'], ['family#3'], ['family#3'], ['family#3'], ['family#6'], ['family#6'], ['family#6'], ['family#6'], ['family#6']]


In [25]:
y_pred = model.predict(X_test_scaled)

outputdf = pd.DataFrame({"SampleName": dog_data_test["sample_name"],"Anonymized_Name":dog_data_test["anonymized_name"],"Prediction": y_pred, "Actual": y_test.ravel()}).reset_index(drop=True)
print(outputdf.shape)
outputdf.head()

(177, 4)


,SampleName,Anonymized_Name,Prediction,Actual
0,797.03.CA.BackLeftPaw,FS_175,family#71,family#3
1,797.03.CA.BackRightPaw,FS_175,family#71,family#3
2,797.03.CA.Forehead,FS_175,family#71,family#3
3,797.03.CA.FrontRightPaw,FS_175,family#3,family#3
4,797.03.CA.Tongue,FS_175,family#71,family#3


In [26]:
sumOutput = outputdf.groupby(["Anonymized_Name","Actual","Prediction"]).count()
sumOutput.reset_index()
sumOutput.head()

SampleName
Anonymized_Name Actual   Prediction            
FS_175          family#3 family#3             1
                         family#71            4
FS_176          family#6 family#17            1
                         family#27            1
                         family#49            1

In [27]:
# sumOutput.groupby(["Anonymized_Name"])['SampleName'].max()
idx = sumOutput.groupby(["Anonymized_Name"])['SampleName'].transform(max) == sumOutput['SampleName']
maxOutput = sumOutput[idx]
maxOutput = maxOutput.reset_index()
print(maxOutput.shape)
# maxOutput

(72, 4)


In [28]:
maxOutput_noDup = maxOutput.drop_duplicates(subset=['Anonymized_Name'], keep="first")
print(maxOutput_noDup.shape)
# maxOutput_noDup

(36, 4)


In [29]:
maxOutput_C = maxOutput_noDup[maxOutput_noDup.Actual == maxOutput_noDup.Prediction]

print(maxOutput_C.shape)
# maxOutput_C

(8, 4)


In [30]:
maxOutput_I = maxOutput_noDup[maxOutput_noDup.Actual != maxOutput_noDup.Prediction]

print(maxOutput_I.shape)
# maxOutput_I

(28, 4)


In [31]:
correct_total = maxOutput_C["Anonymized_Name"].count()
incorrect_total = maxOutput_I["Anonymized_Name"].count()

accuracy_maxed_pred = correct_total/(correct_total + incorrect_total)
print(f' Accuracy of most-predicted (dog) family: {round(accuracy_maxed_pred,3)}')

 Accuracy of most-predicted (dog) family: 0.222
